In [3]:
from dotenv import dotenv_values
config = dotenv_values(".env")

In [4]:
from openai import OpenAI
client = OpenAI(api_key=config["OPENAI_API_KEY"])

In [5]:
# taking inspiration from
# https://cookbook.openai.com/examples/how_to_use_guardrails
# https://earnup.atlassian.net/wiki/spaces/AI/pages/2864218151/Agent+Routing

In [28]:
from pydantic import BaseModel, Field
from openai import OpenAI
from enum import Enum, StrEnum

class Topics(StrEnum):
    personal_financial_guidance="personal_financial_guidance"
    support_assistance="support_assistance"
    debt_management="debt_management"
    budget_management_tracking="budget_management_tracking"
    financial_goals_planning="financial_goals_planning"
    contact_financial_institution="contact_financial_institution"
    other="other"

class TopicDetection(BaseModel):
    topic: list[Topics] = Field(description="the topic of the last query provided by the user in the messages list")
    last_query: str = Field(description="the last query provided by the user in the messages list")
    topic_detection_confidence : float = Field(description="the confidence score associated to the topic detection score between 0 and 1, 0 being not confident, 1 being confident")


In [29]:
topic_list = """
- Personalized Financial Guidance: Tailored advice to help users make informed financial decisions.
- 24/7 Support: Continuous availability to assist users with their financial queries and concerns.
- Debt Management: Tools to help users manage and reduce their debts effectively.
- Budget Management and Tracking: Features to create, manage, and track personal budgets.
- Financial Goals Planning: Assistance in setting and achieving financial goals.
- Integration with Financial Institutions: Seamless connection with banks and financial services for real-time data."""

template_str = f"""
### List of authorized topics ###
{ topic_list }
                
### Instruction: find the topic of the current query. ###

### Additional instructions ###
- use the TopicDetection pydantic model to output your result
"""

In [30]:
print(template_str)


### List of authorized topics ###

- Personalized Financial Guidance: Tailored advice to help users make informed financial decisions.
- 24/7 Support: Continuous availability to assist users with their financial queries and concerns.
- Debt Management: Tools to help users manage and reduce their debts effectively.
- Budget Management and Tracking: Features to create, manage, and track personal budgets.
- Financial Goals Planning: Assistance in setting and achieving financial goals.
- Integration with Financial Institutions: Seamless connection with banks and financial services for real-time data.
                
### Instruction: find the topic of the current query. ###

### Additional instructions ###
- use the TopicDetection pydantic model to output your result



In [60]:
import asyncio

system_prompt_base = "You are a helpful assistant."


def get_chat_response(system_prompt: str, user_request: str):
    """_summary_

    :param str system_prompt: _description_
    :param str user_request: _description_
    :return _type_: _description_
    """
    print("Getting LLM response")
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_request},
    ]
    response = client.chat.completions.create(
        model=config["OPEN_AI_MODEL_4o_MINI"], messages=messages, temperature=0.0
    )
    print("Got LLM response")

    return response.choices[0].message.content

# A common design to minimize latency is to send your guardrails asynchronously along with your main LLM call. 
# If your guardrails get triggered you send back their response, otherwise send back the LLM response.

def topical_guardrail(sytem_prompt: str, user_request: str):
    print("Checking topical guardrail")
    messages = [
        {
            "role": "system",
            "content": sytem_prompt,
        },
        {"role": "user", "content": user_request},
    ]
    response = client.beta.chat.completions.parse(
        model=config["OPEN_AI_MODEL_4o_MINI"], 
        messages=messages, 
        temperature=0,
        response_format=TopicDetection,
    )

    print("Got guardrail response")
    return response


def execute_chat_with_guardrail(query: str):
    # topical_guardrail_task = asyncio.create_task(topical_guardrail(sytem_prompt=template_str, user_request=query))
    # chat_task = asyncio.create_task(get_chat_response(system_prompt=system_prompt_base, user_request=query))
    topical_guardrail_task = topical_guardrail(sytem_prompt=template_str, user_request=query)
    print(topical_guardrail_task.choices[0].message.parsed.model_dump())
    completion_guardrail = topical_guardrail_task.choices[0].message.parsed.model_dump()
    found_topic = completion_guardrail["topic"][0].value
    authorized_topics = [t.value for t in Topics]
    if found_topic != "other":
        print(f"query: {query} is authorized")
    else:
        print(f"query: {query} is not authorized")

In [61]:
# Call the main function with the good request - this should go through
query: str = "I qualify for a hardship withdrawal from my 401K plan, I know that I'll be subject to a penalty but what is the penalty rate?"
response = execute_chat_with_guardrail(query)

Checking topical guardrail
Got guardrail response
{'topic': [<Topics.debt_management: 'debt_management'>], 'last_query': "I qualify for a hardship withdrawal from my 401K plan, I know that I'll be subject to a penalty but what is the penalty rate?", 'topic_detection_confidence': 0.85}
query: I qualify for a hardship withdrawal from my 401K plan, I know that I'll be subject to a penalty but what is the penalty rate? is authorized


In [62]:
query: str = "tell me a joke about cats ?"
response = execute_chat_with_guardrail(query)

Checking topical guardrail
Got guardrail response
{'topic': [<Topics.other: 'other'>], 'last_query': 'tell me a joke about cats ?', 'topic_detection_confidence': 0.1}
query: tell me a joke about cats ? is not authorized
